In [1]:
import torch
from torch_geometric.data import Data
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
import concurrent.futures
import torch
from torch.utils.data import Dataset, ConcatDataset, random_split
from torch_geometric.loader import DataLoader

In [4]:
fileset = {}

sig_dir = '/mdsmlvol/rechits_v4/point_clouds/'
fileset['sample'] = [sig_dir + f'BToKPhi_MuonLLPDecayGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_{str(i).zfill(7)}_point_clouds.pt' for i in range(328)]
# fileset['sample'] = [sig_dir + f'BToKPhi_MuonLLPDecayGenFilter_PhiToPi0Pi0_mPhi0p3_ctau300_{str(i).zfill(7)}_graphs.pt' for i in range(2)]

# bkg_dir = '/ceph/cms/store/user/aaportel/B-Parking/rechits_v2/ParkingBPH1_2018A/'
# fileset['background'] = [bkg_dir + f'ParkingBPH1_2018A_{str(i).zfill(7)}.root' for i in range(380)]

In [19]:
shuffle_dataset = True
random_seed = 42
split_ratio = 0.8  # 80% of the data for training, 20% for testing

# Set random seed for reproducibility
torch.manual_seed(random_seed)

datasets = [torch.load(fp) for fp in fileset['sample']]
dataset = ConcatDataset(datasets)

dataset_size = len(dataset)
train_size = int(split_ratio * dataset_size)
test_size = dataset_size - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])
train_loader = DataLoader(train_dataset, shuffle=shuffle_dataset)
test_loader = DataLoader(test_dataset, shuffle=shuffle_dataset)

In [23]:
import torch
import torch_geometric
from torch_geometric.nn import DynamicEdgeConv, global_mean_pool, BatchNorm

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
settings = {
    "k": 20,
    "input_features": 17,
    "hidden_units": [64, 128, 256],
    "fc_units": [256, 128],
    "dropout": 0.5,
    "output_classes": 2,
}

class DGCNN(torch.nn.Module):
    def __init__(self, settings):
        super(DGCNN, self).__init__()
        self.k = settings['k']
        self.input_bn = BatchNorm(settings['input_features'])

        # DGCNN uses a series of Dynamic Edge Convolutions
        self.conv_layers = torch.nn.ModuleList()
        in_channels = settings['input_features']
        for out_channels in settings['hidden_units']:
            conv = DynamicEdgeConv(
                torch.nn.Sequential(
                    torch.nn.Linear(2 * in_channels, out_channels),
                    BatchNorm(out_channels),
                    torch.nn.ReLU(),
                    torch.nn.Linear(out_channels, out_channels),
                    BatchNorm(out_channels),
                    torch.nn.ReLU()
                ), k=self.k, aggr='max'
            )
            self.conv_layers.append(conv)
            in_channels = out_channels

        # Fully connected layers
        self.fc_layers = torch.nn.ModuleList()
        prev_channels = in_channels
        for units in settings['fc_units']:
            self.fc_layers.append(
                torch.nn.Sequential(
                    torch.nn.Linear(prev_channels, units),
                    torch.nn.ReLU(),
                    torch.nn.Dropout(p=settings['dropout'])
                )
            )
            prev_channels = units

        # Output layer
        self.out = torch.nn.Linear(prev_channels, settings['output_classes'])

    def forward(self, data):
        x, pos, batch = data.x, data.pos, data.batch

        x = self.input_bn(x)
        for conv in self.conv_layers:
            # The EdgeConv layers now take only x and batch as input
            x = conv(x, batch)

        x = global_mean_pool(x, batch)

        for fc in self.fc_layers:
            x = fc(x)

        x = self.out(x)
        return torch.nn.functional.softmax(x, dim=1)



model = DGCNN(settings).to(DEVICE)
print(model)


DGCNN(
  (input_bn): BatchNorm(17)
  (conv_layers): ModuleList(
    (0): DynamicEdgeConv(nn=Sequential(
      (0): Linear(in_features=34, out_features=64, bias=True)
      (1): BatchNorm(64)
      (2): ReLU()
      (3): Linear(in_features=64, out_features=64, bias=True)
      (4): BatchNorm(64)
      (5): ReLU()
    ), k=20)
    (1): DynamicEdgeConv(nn=Sequential(
      (0): Linear(in_features=128, out_features=128, bias=True)
      (1): BatchNorm(128)
      (2): ReLU()
      (3): Linear(in_features=128, out_features=128, bias=True)
      (4): BatchNorm(128)
      (5): ReLU()
    ), k=20)
    (2): DynamicEdgeConv(nn=Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): BatchNorm(256)
      (2): ReLU()
      (3): Linear(in_features=256, out_features=256, bias=True)
      (4): BatchNorm(256)
      (5): ReLU()
    ), k=20)
  )
  (fc_layers): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=256, out_features=256, bias=True)
      (1): ReLU()
 

In [25]:
# Assuming train_loader is already created as per your code

# Get a single batch of data
data = next(iter(train_loader))

# Move data to the same device as the model
data = data.to(DEVICE)

# Forward pass through the model
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    output = model(data)

# Output examination
print("Output shape:", output.shape)
print("Output values:", output)


TypeError: DynamicEdgeConv.forward() takes from 2 to 3 positional arguments but 4 were given